# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [1]:
import xlwings as xw
import pandas as pd
import numpy as np

# xw.__version__

Rebuilding cache of generated files for COM support...
Checking 00020813-0000-0000-C000-000000000046x0x1x9
Could not add module (IID('{00020813-0000-0000-C000-000000000046}'), 0, 1, 9) - <class 'ImportError'>: cannot import name 'Dispatch' from partially initialized module 'win32com.client' (most likely due to a circular import) (C:\Users\Ivan\AppData\Local\Programs\Python\Python310\lib\site-packages\win32com\client\__init__.py)
Done.


In [13]:
max_breads = 7
wb = xw.Book('себестоимостьА_в1.xlsx')
sht = wb.sheets['Рецептура']
rcp_breads = sht.range('рцп_пшеничный_хлеб').value
breads = rcp_breads[3:3+max_breads-1]
bId = -1
for bread in breads:
    bId += 1
    if bread[1]: #  Name is set
        cost = sum([bread[i] * rcp_breads[len(rcp_breads)-3][i] for i in range(5, len(bread)-1) if not bread[i] is None])
        print(bread[1], cost)

Хлеб Венгерский 21.48
Хлеб "Барбари" 16.525
Багет парижский 17.423000000000002
Чиабатта 18.085


2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [2]:
reviews = pd.read_csv('reviews_sample.csv', header=0)
recipes = pd.read_csv('recipes_sample.csv', header=0)
reviews['date'] = pd.to_datetime(reviews['date'])
combined = pd.merge(recipes, reviews, left_on='id', right_on='recipe_id')

# Оставляем только необходимые столбцы
combined = combined[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]
combined.head()

,id,name,minutes,submitted,description,n_ingredients
0,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
2,44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
3,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,NaN
4,67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.

In [3]:
def get5Percent(df):
    return df.sample(frac=0.05, random_state=1)

In [4]:
saveName = 'recipes.xlsx'
try:
    wbR = xw.Book(saveName)
    defaultName = None
    shtRc = wbR.sheets['Рецепты']
    shtRw = wbR.sheets['Отзывы']
except:
    wbR = xw.Book()
    defaultName = wbR.sheets[0].name
    shtRc = wbR.sheets.add('Рецепты')
    shtRw = wbR.sheets.add('Отзывы')

shtRc.clear()
shtRw.clear()

shtRc.range('A1').options(pd.DataFrame, index=False, header=1).value = get5Percent(recipes)
shtRw.range('A1').options(pd.DataFrame, index=False, header=1).value = get5Percent(reviews)

wbR.save(saveName)

if defaultName:
    wbR.sheets[defaultName].delete()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [5]:
saveName = 'recipes.xlsx'
wb = xw.Book(saveName)
sht = wb.sheets['Рецепты']
table = sht.range('A1').options(pd.DataFrame, expand='table').value
table['seconds_assign'] = table['minutes'].apply(lambda x: x * 60)
sht.clear()
sht.range('A1').options(pd.DataFrame, index=False, header=1).value = table
wb.save('recipesSeconds.xlsx')
table

,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,seconds_assign
name,,,,,,,,
fantastic banana fruitcake,78715.0,160.0,89831.0,2003-12-12,13.0,"this cake is very moist, it is the ultimate in...",NaN,9600.0
greek orzo salad w kalamata and feta,174859.0,65.0,141293.0,2006-06-26,NaN,a great summer salad. serve room temperature ...,NaN,3900.0
yet another tater tot casserole,301114.0,70.0,451055.0,2008-04-29,6.0,i like to play with the below basic recipe. o...,8.0,4200.0
crusty parmesan herb zucchini bites,516384.0,35.0,427739.0,2014-06-03,5.0,aunt donna posted this on my facebook wall.,7.0,2100.0
pomegranate duck,95541.0,106.0,29196.0,2004-07-13,NaN,i made a quail recipe similar to this-but toni...,NaN,6360.0
...,...,...,...,...,...,...,...,...
chef flower s cracked wheat pilaf kibrisli bulgur pilavi,256701.0,23.0,223854.0,2007-10-02,NaN,"this is one of my mother's recipe, she was bor...",7.0,1380.0
ultimate quesadillas,177539.0,40.0,86072.0,2006-07-13,6.0,saving this here for future use! sound quick a...,4.0,2400.0
sausage quiche squares,96295.0,40.0,103876.0,2004-07-24,9.0,this is a great hot appetizer recipe. i made t...,NaN,2400.0


4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [12]:
saveName = 'recipesSeconds.xlsx'
wb = xw.Book(saveName)
sht = wb.sheets['Рецепты']
table = sht.range('A1').options(convert=pd.DataFrame, expand='table').value

s0 = table.shape[0]+2
s1 = table.shape[1]+2

sht.range((1, s1)).value = 'seconds_formula'
for i in range(2, s1):
    sht.range((i, s1)).formula = '=' + sht.range((i, 3)).address + ' * 60'

wb.save('recipesFormula.xlsx')

(1500, 7)
9


5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [13]:
# from xlwings.constants import Constants
from xlwings.constants import VAlign, HAlign

saveName = 'recipesFormula.xlsx'
wb = xw.Book(saveName)
sht: xw.Sheet = wb.sheets['Рецепты']
# rng: xw.Range = sht.range('A1').options(pd.DataFrame, expand='right')
rng: xw.Range = sht.range('A1:Z1')
for cell in rng:
    if cell.value and 'seconds' in cell.value:
        cell.font.bold = True
        cell.api.HorizontalAlignment = HAlign.xlHAlignCenter
        cell.api.VerticalAlignment = VAlign.xlVAlignCenter

wb.save('recipesFormula.xlsx')

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [14]:
saveName = 'recipesFormula.xlsx'
wb = xw.Book(saveName)
sht: xw.Sheet = wb.sheets['Рецепты']
rng: xw.Range = sht.range('A1:Z1')
for cell in rng:
    if cell.value == 'minutes':
#         print(sht.range((1, cell.column)).options(expand='down'))
        for i in range(2, 1600):
            sub_cell = sht.range((i, cell.column))
            try:
                v = int(sub_cell.value)
            except TypeError:
                continue
            if v <= 5:
                sub_cell.color = '#63ff90'
            elif 5 < v < 10:
                sub_cell.color = '#63ff90'
            elif v >= 10:
                sub_cell.color = '#ff6363'
        break

wb.save('recipesFormulaColors.xlsx')

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [34]:
saveName = 'recipesFormulaColors.xlsx'
wb = xw.Book(saveName)
sht = wb.sheets['Рецепты']
table = sht.range('A1').options(convert=pd.DataFrame, expand='table').value

s0 = table.shape[0]+2
s1 = table.shape[1]+2

sht_reviews = wb.sheets[0]
sht_recipes = wb.sheets[1]

sht.range((1, s1)).value = 'n_reviews'
for i in range(2, s0):
    sht.range((i, s1)).formula = f'=COUNTIF({sht_reviews.name}!C:C,{sht_recipes.name}!A{i})'

wb.save('recipesFormulaColorsNReviews.xlsx')

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 